# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imresize
from skimage import io
import datetime
import glob
import keras
import matplotlib.pyplot as plt
from skimage.transform import rescale
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, Dropout, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import *


Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)


In [3]:
DATA_SET_PATH = "./Project_data"

In [4]:

OuterPath = os.path.join (DATA_SET_PATH, "train", "*")
print (OuterPath)

trainImgsFolderPaths = glob.glob(OuterPath)


./Project_data/train/*


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open(DATA_SET_PATH + '/train.csv').readlines())
val_doc = np.random.permutation(open(DATA_SET_PATH +'/val.csv').readlines())


In [6]:
print (train_doc[0:5])
print (val_doc[0:5])

['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'
 'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n']
['WIN_20180926_17_24_20_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'
 'WIN_20180907_15_54_30_Pro_Thumbs Up_new;Thumbs Up_new;4\n'
 'WIN_20180926_17_08_11_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180925_18_01_40_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180907_16_16_48_Pro_Stop Gesture_new;Stop Gesture_new;2\n']


In [7]:
import pandas as pd

def getAblationSequencesDistribution(ablation, folder_list):
    df = pd.DataFrame(folder_list, columns=['folderInfo'])
    df = df.folderInfo.str.strip().str.split(';', expand=True)
    df.columns = ['folderPath', 'gestureType', 'gestureLabel']
    

    outFolderList = (df[df['gestureLabel'].isin(['0'])].head(ablation//5).apply(';'.join, axis=1).tolist())
    outFolderList.extend(df[df['gestureLabel'].isin(['1'])].head(ablation//5).apply(';'.join, axis=1).tolist())
    outFolderList.extend(df[df['gestureLabel'].isin(['2'])].head(ablation//5).apply(';'.join, axis=1).tolist())
    outFolderList.extend(df[df['gestureLabel'].isin(['3'])].head(ablation//5).apply(';'.join, axis=1).tolist())
    outFolderList.extend(df[df['gestureLabel'].isin(['4'])].head(ablation//5).apply(';'.join, axis=1).tolist())
    
    
    return outFolderList
    
getAblationSequencesDistribution(10, val_doc)

['WIN_20180925_18_01_40_Pro_Left_Swipe_new;Left_Swipe_new;0',
 'WIN_20180926_16_46_22_Pro_Left_Swipe_new;Left_Swipe_new;0',
 'WIN_20180907_15_55_06_Pro_Right Swipe_new;Right Swipe_new;1',
 'WIN_20180926_17_24_12_Pro_Right_Swipe_new;Right_Swipe_new;1',
 'WIN_20180926_17_08_11_Pro_Stop_new;Stop_new;2',
 'WIN_20180907_16_16_48_Pro_Stop Gesture_new;Stop Gesture_new;2',
 'WIN_20180926_17_24_20_Pro_Thumbs_Down_new;Thumbs_Down_new;3',
 'WIN_20180907_16_33_15_Pro_Thumbs Down_new;Thumbs Down_new;3',
 'WIN_20180907_15_54_30_Pro_Thumbs Up_new;Thumbs Up_new;4',
 'WIN_20180907_15_38_42_Pro_Thumbs Up_new;Thumbs Up_new;4']

In [8]:
#getAblationSequences(train_doc)

IMAGE_DIM = 120
IMAGE_SEQUENCE = 30
IMAGE_CHANNEL = 3
BATCH_SIZE = 64
NUM_CLASSES = 5

# Scaling factor to scale the image to size of 120
SCALING_FACTOR_120 = 1 
SCALING_FACTOR_160 = 3/4
SCALING_FACTOR_360 = 1/3

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:
def generator(source_path, folder_list, batch_size, ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = (np.arange(0, 30, 1)) 
    
    if (ablation is not None):
        folder_list = getAblationSequencesDistribution(ablation, folder_list)

    while True:
        t = np.random.permutation(folder_list)
        # calculate the number of batches
        num_batches = len(folder_list) // batch_size 
        
        # we iterate over the number of batches
        for batch in range(num_batches): 
            batch_data = np.zeros((batch_size, IMAGE_SEQUENCE, IMAGE_DIM, IMAGE_DIM, IMAGE_CHANNEL)) 
            # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is
            # the number of channels RGB
            
            # batch_labels is the one hot representation of the output
            batch_labels = np.zeros((batch_size, NUM_CLASSES)) 
            
            # iterate over the batch_size
            for folder in range(batch_size): 
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                # read all the images in the folder
                
                #  Iterate over the frames/images of a folder to read them in
                for idx,item in enumerate(img_idx): 
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).\
                                astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == (120,160,3)):
                        image = rescale (image, [SCALING_FACTOR_120, SCALING_FACTOR_160], multichannel=True, mode="constant")
                        
                    else:
                        image = rescale (image, [SCALING_FACTOR_360,SCALING_FACTOR_360], multichannel=True, mode="constant")
                        
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min()) / (image[:,:,0].max() - \
                                                        image[:,:,0].min())
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min()) / (image[:,:,1].max() - \
                                                        image[:,:,1].min())
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min()) / (image[:,:,2].max() - \
                                                        image[:,:,2].min())
                    #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            #you yield the batch_data and the batch_labels, remember what does yield do    
            yield batch_data, batch_labels 

        
        # write the code for the remaining data points which are left after full batches
        
        remainingSequence = (len(folder_list) % batch_size)
        if (remainingSequence != 0):
            batch_data = np.zeros((remainingSequence,IMAGE_SEQUENCE, IMAGE_DIM,IMAGE_DIM, IMAGE_CHANNEL)) 
            # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 
            #is the number of channels RGB
            
            batch_labels = np.zeros((remainingSequence, NUM_CLASSES)) 
            # batch_labels is the one hot representation of the output
            
            startFolderPathIdx = len(folder_list) - remainingSequence
            for folder in range(remainingSequence):
                imgs = os.listdir(source_path+'/'+t[startFolderPathIdx + folder].split(';')[0])
                
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+t[startFolderPathIdx + folder].strip().split(';')[0]+'/'+imgs[item]).\
                                    astype(np.float32)
                    
                    if (image.shape == (120, 160, 3)):
                        image = rescale(image, [SCALING_FACTOR_120,SCALING_FACTOR_160], multichannel=True, mode="constant")
                    else:
                        image = rescale(image, [SCALING_FACTOR_360, SCALING_FACTOR_360], multichannel=True, mode="constant")
                        
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min()) / (image[:,:,0].max() - \
                                                                image[:,:,0].min()) 
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min()) / (image[:,:,1].max() - \
                                                                image[:,:,1].min()) 
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min()) / (image[:,:,2].max() - \
                                                                image[:,:,2].min()) 
                    #normalise and feed in the image
                
                batch_labels[folder, int(t[startFolderPathIdx + folder].strip().split(';')[2])] = 1
            
            yield batch_data, batch_labels
    


In [10]:
DATA_SET_PATH = "./Project_data/"
curr_dt_time = datetime.datetime.now()
train_path = DATA_SET_PATH + 'train'
val_path = DATA_SET_PATH + 'val'

In [11]:
testGen = generator(train_path, train_doc, 1)
testValGen = generator(val_path, val_doc, 1)
next(testGen)
next(testValGen)

Source path =  ./Project_data/train ; batch size = 1


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Source path =  ./Project_data/val ; batch size = 1


(array([[[[[0.91764706, 0.87843137, 0.87058824],
           [1.        , 0.99607843, 1.        ],
           [0.98823529, 0.98823529, 0.99607843],
           ...,
           [0.57254902, 0.5372549 , 0.47058824],
           [0.58039216, 0.54901961, 0.4745098 ],
           [0.6       , 0.54509804, 0.49411765]],
 
          [[0.74117647, 0.7254902 , 0.71372549],
           [0.80784314, 0.78823529, 0.76470588],
           [0.89411765, 0.87843137, 0.83137255],
           ...,
           [0.59215686, 0.54509804, 0.49019608],
           [0.5254902 , 0.47843137, 0.41568627],
           [0.56862745, 0.5254902 , 0.44705882]],
 
          [[0.69411765, 0.68235294, 0.65490196],
           [0.71764706, 0.70196078, 0.65490196],
           [0.78039216, 0.75686275, 0.70196078],
           ...,
           [0.57254902, 0.51764706, 0.46666667],
           [0.5254902 , 0.47843137, 0.41568627],
           [0.57647059, 0.54509804, 0.4627451 ]],
 
          ...,
 
          [[0.        , 0.01176471, 0.007843

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

def get3dConvModel(dropOutVal):
    #write your model here , add padding='same' for not decreasing the image size. 
    
    print ("using dropout val",dropOutVal)
    model = Sequential()
    model.add(Conv3D(filters=8, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same',\
                     input_shape= (IMAGE_SEQUENCE, IMAGE_DIM, IMAGE_DIM, IMAGE_CHANNEL), data_format='channels_last'))
    
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    
    model.add(Conv3D(filters=16, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Conv3D(filters=32, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(filters=64, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    model.add(BatchNormalization())
 
    model.add(Flatten())
    model.add(Dense(256, activation='relu')) #kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu')) #kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    return model

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
dropOutList = [0.25]

models = []
modelIdx = 0
for dropOut in dropOutList:
    models.append(get3dConvModel(dropOut))
    optimiser = optimizers.Adam()
    models[modelIdx].compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (models[modelIdx].summary())
    modelIdx += 1

using dropout val 0.25
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 7, 30, 30, 16)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 30, 30, 16)     64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 30, 30, 16)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 30, 30, 32)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.0, verbose=1)
# write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

In [15]:
num_train_sequences_ablation = 663
num_val_sequences_ablation = 100
num_epochs_ablation = 30

The steps_per_epoch and validation_steps are used by fit_generator to decide the number of next() calls it need to make.

In [16]:
train_generator_ablation = generator(train_path, train_doc, BATCH_SIZE) #, ablation=num_train_sequences_ablation)
val_generator_ablation = generator(val_path, val_doc, BATCH_SIZE)#, ablation=num_val_sequences_ablation)

In [17]:
if (num_train_sequences_ablation%BATCH_SIZE) == 0:
    steps_per_epoch_ablation = int(num_train_sequences_ablation/BATCH_SIZE)
else:
    steps_per_epoch_ablation = (num_train_sequences_ablation//BATCH_SIZE) + 1

if (num_val_sequences_ablation%BATCH_SIZE) == 0:
    validation_steps_ablation = int(num_val_sequences_ablation/BATCH_SIZE)
else:
    validation_steps_ablation = (num_val_sequences_ablation//BATCH_SIZE) + 1

print ("steps_per_epoch_ablation", steps_per_epoch_ablation)
print ("validation_steps_ablation", validation_steps_ablation)
print ("num_train_sequences_ablation", num_train_sequences_ablation)
print ("num_val_sequences_ablation", num_val_sequences_ablation)

steps_per_epoch_ablation 11
validation_steps_ablation 2
num_train_sequences_ablation 663
num_val_sequences_ablation 100


In [18]:
for model in models:

    model.fit_generator(train_generator_ablation, steps_per_epoch=steps_per_epoch_ablation, epochs=num_epochs_ablation,
                        verbose=1, 
                        callbacks=callbacks_list, validation_data=val_generator_ablation, 
                        validation_steps=validation_steps_ablation, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train Epoch 1/30
; batch size = 64


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


11/11 [==============================] - 277s 25s/step - loss: 1.8967 - categorical_accuracy: 0.3261 - val_loss: 1.2735 - val_categorical_accuracy: 0.5500

Epoch 00001: saving model to model_init_2020-07-0115_39_18.722839/model-00001-1.90609-0.32730-1.27346-0.55000.h5
Epoch 2/30
11/11 [==============================] - 128s 12s/step - loss: 1.4736 - categorical_accuracy: 0.4258 - val_loss: 1.3038 - val_categorical_accuracy: 0.4600

Epoch 00002: saving model to model_init_2020-07-0115_39_18.722839/model-00002-1.48313-0.42534-1.30384-0.46000.h5
Epoch 3/30
11/11 [==============================] - 136s 12s/step - loss: 1.4182 - categorical_accuracy: 0.4324 - val_loss: 1.0773 - val_categorical_accuracy: 0.5700

Epoch 00003: saving model to model_init_2020-07-0115_39_18.722839/model-00003-1.43033-0.42986-1.07729-0.57000.h5
Epoch 4/30
11/11 [==============================] - 137s 12s/step - loss: 1.1890 - categorical_accuracy: 0.5399 - val_loss: 1.5327 - val_categorical_accuracy: 0.5500

Epoc

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
# The models are generated and we will pick the best model based on the best epoch.

Inference : Epoch 21 has the best balanced Training and validation percentage. The loss is also at absorbing level. 

We have the categorical_accuracy: 0.7933 , val_loss: 0.7799 and the val_categorical_accuracy: 0.7300
So this will be our model for finalization.